<a href="https://colab.research.google.com/github/lguloy/House-Pricing-Predictions/blob/House_Price_ML_CCG/House_Price_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:

from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/all_dumimies_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("all_dumimies_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
df = df.dropna(how="any")

+---+----------+-----------+-------+-----------+-----------+---------+------------+----------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+------------+----------+-----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+---------+-----------+-----------+-----------+-----------+-----------+----------+------------+------------+------------+---------------+---------------+---------------+----------------+-----------------+-------------+-------------+----------------+-------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------------------+-------------------+--------------------+-----------------

In [4]:
#Get the Feature names
import pandas as pd

features = [col for col in df.columns]
print(features)

print(type(features))

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave', 'Alley_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_NoSeWa', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_

In [5]:
#Import sklearn tools and dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [6]:
# Assign the data to X and y

X1 = df[["1stFlrSF", "GrLivArea", "LotArea", "GarageArea", "BsmtUnfSF", "TotalBsmtSF", "LotFrontage", "GarageYrBlt", "MoSold", "YearBuilt"]]
y1 = df["SalePrice"]
print(X1.shape, y1.shape)

(1121, 10) (1121,)


In [7]:
# Use train_test_split to create training and testing data

### BEGIN SOLUTION
from sklearn.model_selection import train_test_split

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=20, test_size=0.20)

### END SOLUTION

In [8]:
# Create the model using RandomForestRegressor

### BEGIN SOLUTION
from sklearn.ensemble import RandomForestRegressor
model_1 = RandomForestRegressor()
### END SOLUTION

In [10]:
# Fit the model to the training data and calculate the scores for the training and testing data

### BEGIN SOLUTION
model_1.fit(X1_train, y1_train)
training_score_1 = model_1.score(X1_train, y1_train)
testing_score_1 = model_1.score(X1_test, y1_test)

### END SOLUTION 

print(f"Training Score: {training_score_1}")
print(f"Testing Score: {testing_score_1}")

Training Score: 0.9741194354626913
Testing Score: 0.8185892858574557


In [11]:
# Assign the data to X and y

X2 = df[["1stFlrSF", 
        "GrLivArea", 
        "LotArea", 
        "GarageArea", 
        "BsmtUnfSF", 
        "TotalBsmtSF", 
        "LotFrontage", 
        "GarageYrBlt", 
        "MoSold", 
        "YearBuilt",
        "YearRemodAdd",
        "BsmtFinSF1",
        "OpenPorchSF",
        "WoodDeckSF",
        "TotRmsAbvGrd",
        "YrSold",
        "MasVnrArea",
        "2ndFlrSF",
        "OverallQual",
        "BedroomAbvGr"
        ]]
y2 = df["SalePrice"]
print(X2.shape, y2.shape)

(1121, 20) (1121,)


In [12]:
# Use train_test_split to create training and testing data

### BEGIN SOLUTION
from sklearn.model_selection import train_test_split

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=20, test_size=0.20)

### END SOLUTION

In [13]:
# Create the model using RandomForestRegressor

### BEGIN SOLUTION
from sklearn.ensemble import RandomForestRegressor
model_2 = RandomForestRegressor()
### END SOLUTION

In [15]:
# Fit the model to the training data and calculate the scores for the training and testing data

### BEGIN SOLUTION
model_2.fit(X2_train, y2_train)
training_score_2 = model_2.score(X2_train, y2_train)
testing_score_2 = model_2.score(X2_test, y2_test)

### END SOLUTION 

print(f"Training Score: {training_score_2}")
print(f"Testing Score: {testing_score_2}")

Training Score: 0.9763409902072882
Testing Score: 0.8816466480361129
